In [1]:
import sys

sys.path.append('..')
import os

os.chdir('..')

import pandas as pd
pd.set_option('display.max_rows' , 500)
pd.set_option('display.max_columns' , 200)



In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import math
import random
import awswrangler as wr
import os
import warnings
warnings.simplefilter(action='ignore')
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as stats
from statsmodels.stats.weightstats import ztest

pd.set_option('display.max_rows' , 500)
pd.set_option('display.max_columns' , 200)
pd.set_option('display.max_colwidth' , 100)

plt.rcParams['font.sans-serif'] = ['Taipei Sans TC Beta']

In [5]:
%reload_ext autoreload
%autoreload 2


In [3]:
from sample.method.other_function import event_map_to_query
from sample.data_prep.data_extract import Dataset


In [4]:
var_dict = {0:'AA' , 1:'A' , 2:'B' , 3:'C' , 4:'D' , 5:'E'}
def style_row(row):
    if row.name % 2 == 0:
        if row.name ==4:
            return ['background-color: lavender ; color:firebrick;font-weight: bold' for _ in row]
        # 偶数行设置为深蓝色
        return ['background-color: lavender ; color:black' for _ in row]
    else:
        # 奇数行设置为浅蓝色
        return ['background-color: ghostwhite;color:black' for _ in row]




def exp_results(df,column ,test_var,test_type = 0 , device = 'all',var_name = var_dict , merge_aa = True):
    name_dict = {f'all_{column}':column ,'index':'',f'device_{column}':column}
    
    if device == 'all':
        df = df.rename({column:f'all_{column}'} , axis =1)
        temp = df.groupby(['var_id' , 'beacon_uid'])[f'all_{column}'].mean().groupby('var_id').mean()
    else:
        df = df.rename({column:f'device_{column}'} , axis =1)
        temp = df.query('device == @device').groupby(['var_id' , 'beacon_uid'])[f'device_{column}'].mean().groupby('var_id').mean()

    df_metric = pd.DataFrame(metric_test(df ,test_var , device = device ,metrics_list = [column], significant_level = .05 , merge_aa = merge_aa))
    df_metric.loc['mean_diff%'] = df_metric.loc['mean_diff']/df_metric.loc['group_1_mean']
    df_metric = df_metric.loc[['group_1_mean' , 'group_2_mean' ,'mean_diff', 'mean_diff%','is_significant','p_value']].rename({'group_1_mean':f'var_{var_name[test_var[0]]}','group_2_mean':f'var_{var_name[test_var[1]]}'} )
    
    if test_type == 0:
        display(pd.concat([df_metric.iloc[:4].applymap(lambda x:'{:.1%}'.format(x)) , df_metric.iloc[4:]]).reset_index().rename(name_dict,axis =1).style.apply(style_row, axis=1))
    else:
        display(pd.concat([df_metric.iloc[:3].applymap(lambda x:'{:.2f}'.format(x)),df_metric.iloc[3].to_frame().T.applymap(lambda x:'{:.1%}'.format(x)) , df_metric.iloc[4:]] , axis = 0).reset_index().rename(name_dict,axis =1).style.apply(style_row, axis=1))
    
    print(device)
    return temp
    
    

# Pre Test

In [5]:
from sample.method.other_function import event_map_to_query
from sample.data_prep.data_extract import Dataset
from sample.data_prep.data_pre_tests import srm_check
from sample.data_prep.data_pre_tests import metric_test

from sample.method.hypothesis_testing import chi_square_test

## view_search_restuls with filter on material 或者是 freeshiping  的 User 從 srp 點到 item 且待超過 20s 的數量

In [7]:
data_ = Dataset('beq_video_content_exp' , '2024-8-4' , '2024-8-18' , 
                # exp_start_date='2024-7-17',
                 device = ['android' , 'iphone'] , 
                 version= '6.66.1',
                # lang=['zh_TW'],
                # geo = 'TW',
                trigger = {
                           'view_search_results':
                           {'filters_dict':
                            {'dict_key':'material','cal':'contains','value':'gold'},
                            'filters_dict_':
                            {'dict_key':'free_shipping','cal':'=','value':'1'}
                           }
                          }
                )

print(data_.get_raw_data(mode = 'sql'))
# raw_data = data_.get_raw_data(mode = 'data')
# exp_data = data_.get_exp_data('data')



        
                
                WITH expt AS (
                    SELECT
                        beacon,
                        exp_id,
                        var_id,
                        date(created) AS created_date
                    FROM
                        ath_experiment
                    WHERE
                        exp_id = 'beq_video_content_exp'
                        AND date(created) <= date '2024-8-18'
                        
                ),
            
                
                buyer_journey AS (
                    SELECT
                        visit_date,
                        first_session_timestamp,
                        date(
                            at_timezone(
                                first_session_timestamp,
                                'Asia/Taipei'
                            )
                        ) AS session_date,
                        beacon_uid,
                        b.beacon AS beacon,
        

In [46]:
print(data_.get_metric_data(mode= 'sql' ,
                     metric = {
                         'metric_main':{
                             'metric_name':'avg.view_from_',
                             'source':'tracking',
                             'group_level':'user',
                             'method':'count_distinct',
                             'condition':{'view_item':
                                          {'from_screen':'search_results',
                                          'current_timer':{'cal':'>=' , 'value':'20'}}
                                         },
                             'metric':{'view_item':'view_id'}
                         }    
                              }))

WITH expt AS
  (SELECT beacon,
          exp_id,
          var_id,
          date(created) AS created_date
   FROM ath_experiment
   WHERE exp_id = 'beq_video_content_exp'
     AND date(created) <= date '2024-8-18' ),
     buyer_journey AS
  (SELECT visit_date,
          first_session_timestamp,
          date(at_timezone(first_session_timestamp, 'Asia/Taipei')) AS session_date,
          beacon_uid,
          b.beacon AS beacon,
          b.device AS device,
          b.session_duration_sec session_duration,
          geo,
          max_member_status,
          is_seller,
          first_session_timestamp,
          funnel_e1,
          funnel_e2,
          funnel_interest,
          funnel_consideration,
          add_to_cart,
          funnel_purchase,
          e2__uniq_items,
          if(funnel ['view_item'] IS NULL, 0, funnel ['view_item'].event_cnt) view_item_cnt,
          e2__session_duration,
          interest__shop_list,
          interest__item_list,
          search__ter

# view home，且 impression style pick 或 impression 第 5 個 section 的 User ，點擊 style pick 後到達的 item & info_dict 中的 utm_term 是 dress，進一步從 middle/bottom similar item 往後走的比例 

In [50]:
data_ = Dataset('beq_video_content_exp' , '2024-8-4' , '2024-8-18' , 
                # exp_start_date='2024-7-17',
                 device = ['android' , 'iphone'] , 
                 version= '6.66.1',
                # lang=['zh_TW'],
                # geo = 'TW',
                trigger = {'view_home':None,
                           'impression_section':(
                               {'screen_name':'home','section':['homepage-style-pick1' , 'homepage-style-pick2']},
                               {'screen_name':'home','position':'5'}
                           ),
                          }
                )

# print(data_.get_raw_data(mode = 'sql'))
# raw_data = data_.get_raw_data(mode = 'data')
# exp_data = data_.get_exp_data('data')


In [94]:
print(data_.get_metric_data(mode= 'sql' ,
                     metric = {
                         'metric_trigger':{
                             'source':'tracking',
                             'param':{
                                 'view_item':{
                                     'from_screen':'home' , 
                                     'from_section':['homepage-style-pick1' , 'homepage-style-pick2'],
                                     'info_dict':{'dict_key':'utm_term' , 'value':'dress'}
                                 }
                                     }
                             
                             
                         },
                         'metric_main':{
                             'metric_name':'avg.view_from_item_from_stylepick',
                             'source':'tracking',
                             'group_level':'event',
                             'method':'boolean',
                             'condition':{'view_item':
                                          {'from_screen':'item',
                                          'from_section':['middle_similar_item' , 'bottom_similar_item']}
                                         },
                             'metric':{'view_item':'view_id'}
                         }    
                              }))

view_item
WITH expt AS
  (SELECT beacon,
          exp_id,
          var_id,
          date(created) AS created_date
   FROM ath_experiment
   WHERE exp_id = 'beq_video_content_exp'
     AND date(created) <= date '2024-8-18' ),
     buyer_journey AS
  (SELECT visit_date,
          first_session_timestamp,
          date(at_timezone(first_session_timestamp, 'Asia/Taipei')) AS session_date,
          beacon_uid,
          b.beacon AS beacon,
          b.device AS device,
          b.session_duration_sec session_duration,
          geo,
          max_member_status,
          is_seller,
          first_session_timestamp,
          funnel_e1,
          funnel_e2,
          funnel_interest,
          funnel_consideration,
          add_to_cart,
          funnel_purchase,
          e2__uniq_items,
          if(funnel ['view_item'] IS NULL, 0, funnel ['view_item'].event_cnt) view_item_cnt,
          e2__session_duration,
          interest__shop_list,
          interest__item_list,
          s

# general metric

In [6]:
data_ = Dataset('beq_video_content_exp' , '2024-8-4' , '2024-8-18' , 
                # exp_start_date='2024-7-17',
                 device = ['android' , 'iphone'] , 
                 version= '6.66.1',
                # lang=['zh_TW'],
                # geo = 'TW',
                trigger = {'view_feed':None,
                          }
                )

# print(data_.get_raw_data(mode = 'sql'))
# raw_data = data_.get_raw_data(mode = 'data')
# exp_data = data_.get_exp_data('data')


## funnel

In [98]:
journey_metric_list = [
'funnel_e1', 'funnel_e2',
'funnel_interest', 'funnel_consideration',
'add_to_cart', 'funnel_purchase',
]

start_funnel = 'V'
end_funnel = 'add_to_cart'


if start_funnel == 'V':
    final = data_.get_metric_data(mode= 'sql' ,
                     metric = {
                         'metric_main':{
                             'metric_name':f'V->{end_funnel}',
                             'source':'journey',
                             'group_level':'user',
                             'method':'max',
                             'metric':{end_funnel:None}
                         }    
                              })
else:
    final = data_.get_metric_data(mode= 'sql' ,
                         metric = {
                             'metric_trigger':{
                                 'source':'journey',
                                 'param':{start_funnel:None}


                             },
                             'metric_main':{
                                 'metric_name':f'{start_funnel} -> {end_funnel}',
                                 'source':'journey',
                                 'group_level':'user',
                                 'method':'max',
                                 'metric':{end_funnel:None}
                             }    
                                  })
    
print(final)

WITH expt AS
  (SELECT beacon,
          exp_id,
          var_id,
          date(created) AS created_date
   FROM ath_experiment
   WHERE exp_id = 'beq_video_content_exp'
     AND date(created) <= date '2024-8-18' ),
     buyer_journey AS
  (SELECT visit_date,
          first_session_timestamp,
          date(at_timezone(first_session_timestamp, 'Asia/Taipei')) AS session_date,
          beacon_uid,
          b.beacon AS beacon,
          b.device AS device,
          b.session_duration_sec session_duration,
          geo,
          max_member_status,
          is_seller,
          first_session_timestamp,
          funnel_e1,
          funnel_e2,
          funnel_interest,
          funnel_consideration,
          add_to_cart,
          funnel_purchase,
          e2__uniq_items,
          if(funnel ['view_item'] IS NULL, 0, funnel ['view_item'].event_cnt) view_item_cnt,
          e2__session_duration,
          interest__shop_list,
          interest__item_list,
          search__ter

## avg.Session Duration per Day

In [100]:
final = data_.get_metric_data(mode= 'sql' ,
                 metric = {
                     'metric_main':{
                         'metric_name':f'Avg.Session_duration per Day',
                         'source':'journey',
                         'group_level':'day',
                         'method':'sum',
                         'metric':{'session_duration':None}
                     }    
                          })
print(final)

WITH expt AS
  (SELECT beacon,
          exp_id,
          var_id,
          date(created) AS created_date
   FROM ath_experiment
   WHERE exp_id = 'beq_video_content_exp'
     AND date(created) <= date '2024-8-18' ),
     buyer_journey AS
  (SELECT visit_date,
          first_session_timestamp,
          date(at_timezone(first_session_timestamp, 'Asia/Taipei')) AS session_date,
          beacon_uid,
          b.beacon AS beacon,
          b.device AS device,
          b.session_duration_sec session_duration,
          geo,
          max_member_status,
          is_seller,
          first_session_timestamp,
          funnel_e1,
          funnel_e2,
          funnel_interest,
          funnel_consideration,
          add_to_cart,
          funnel_purchase,
          e2__uniq_items,
          if(funnel ['view_item'] IS NULL, 0, funnel ['view_item'].event_cnt) view_item_cnt,
          e2__session_duration,
          interest__shop_list,
          interest__item_list,
          search__ter

## avg.view_item_cnt per User

In [101]:
final = data_.get_metric_data(mode= 'sql' ,
                 metric = {
                     'metric_main':{
                         'metric_name':f'Avg.View_Item_cnt per User',
                         'source':'tracking',
                         'group_level':'user',
                         'method':'count_distinct',
                         'metric':{'view_item':'view_id'}
                     }    
                          })
print(final)

WITH expt AS
  (SELECT beacon,
          exp_id,
          var_id,
          date(created) AS created_date
   FROM ath_experiment
   WHERE exp_id = 'beq_video_content_exp'
     AND date(created) <= date '2024-8-18' ),
     buyer_journey AS
  (SELECT visit_date,
          first_session_timestamp,
          date(at_timezone(first_session_timestamp, 'Asia/Taipei')) AS session_date,
          beacon_uid,
          b.beacon AS beacon,
          b.device AS device,
          b.session_duration_sec session_duration,
          geo,
          max_member_status,
          is_seller,
          first_session_timestamp,
          funnel_e1,
          funnel_e2,
          funnel_interest,
          funnel_consideration,
          add_to_cart,
          funnel_purchase,
          e2__uniq_items,
          if(funnel ['view_item'] IS NULL, 0, funnel ['view_item'].event_cnt) view_item_cnt,
          e2__session_duration,
          interest__shop_list,
          interest__item_list,
          search__ter

## avg.view_event_cnt per User

In [8]:
event_list = ['view_home'

    , 'view_search_results'

    , 'view_shop'

    , 'view_item_recommend'

    , 'view_favlist_item'

    , 'view_feed'

    , 'view_topic'

    , 'view_favlist_shop'

    , 'view_flagship'

    , 'view_middle_layer'

    , 'view_favlist_collection'

    , 'view_collection'

    , 'view_video_list'
              
    , 'view_discover'
]


condition_ = {}
for e in event_list:
    condition_[e] = None

# final = data_.get_metric_data(mode= 'sql' ,
#                  metric = {
#                      'metric_main':{
#                          'metric_name':f'Avg.View_Event_cnt per User',
#                          'source':'tracking',
#                          'group_level':'user',
#                          'method':'count_distinct',
#                          'condition':condition_,
#                          'metric':{'view_search_results':'view_id'}
#                      }    
#                           })
# print(final)

data_.get_metric_data(mode= 'data' ,
                 metric = {
                     'metric_main':{
                         'metric_name':f'Avg.View_Event_cnt per User',
                         'source':'tracking',
                         'group_level':'user',
                         'method':'count_distinct',
                         'condition':condition_,
                         'metric':{'view_search_results':'view_id'}
                     }    
                          })

,exp_id,var_id,beacon_uid,device,device_avg.view_event_cnt per user,all_avg.view_event_cnt per user
0,beq_video_content_exp,0,gg_105956196428588888324,iphone,6,6
1,beq_video_content_exp,2,gg_115197245928432010031,android,6,6
2,beq_video_content_exp,0,pp_abjzqznjxqbmtogxlibee1fjdwaj,iphone,8,8
3,beq_video_content_exp,1,fb_7181664961851588,iphone,97,97
4,beq_video_content_exp,0,pp_abzwqvcgvhsdsyehv7ftfjmb_xum,iphone,12,12
...,...,...,...,...,...,...
25880,beq_video_content_exp,0,fb_2752097904819290,iphone,11,11
25881,beq_video_content_exp,0,jackykaka,iphone,15,15
25882,beq_video_content_exp,2,pp_ablys57yryfvqtwatswozk3wwqgy,iphone,17,17
25883,beq_video_content_exp,2,gg_101673389859764900675,iphone,6,6


In [10]:
{
                     'metric_main':{
                         'metric_name':f'Avg.View_Event_cnt per User',
                         'source':'tracking',
                         'group_level':'user',
                         'method':'count_distinct',
                         'condition':condition_,
                         'metric':{'view_search_results':'view_id'}
                     }    
                          }

{'metric_main': {'metric_name': 'Avg.View_Event_cnt per User',
  'source': 'tracking',
  'group_level': 'user',
  'method': 'count_distinct',
  'condition': {'view_home': None,
   'view_search_results': None,
   'view_shop': None,
   'view_item_recommend': None,
   'view_favlist_item': None,
   'view_feed': None,
   'view_topic': None,
   'view_favlist_shop': None,
   'view_flagship': None,
   'view_middle_layer': None,
   'view_favlist_collection': None,
   'view_collection': None,
   'view_video_list': None,
   'view_discover': None},
  'metric': {'view_search_results': 'view_id'}}}

## avg.view_[]_cnt per User

In [117]:
event_name = 'view_topic'
page_name = str.capitalize(str.replace(event_name , 'view_' , ''))
final = data_.get_metric_data(mode= 'sql' ,
                 metric = {
                     'metric_main':{
                         'metric_name':f'Avg.{page_name}_cnt per User',
                         'source':'tracking',
                         'group_level':'user',
                         'method':'count_distinct',
                         'metric':{event_name:'view_id'}
                     }    
                          })
print(final)

WITH expt AS
  (SELECT beacon,
          exp_id,
          var_id,
          date(created) AS created_date
   FROM ath_experiment
   WHERE exp_id = 'beq_video_content_exp'
     AND date(created) <= date '2024-8-18' ),
     buyer_journey AS
  (SELECT visit_date,
          first_session_timestamp,
          date(at_timezone(first_session_timestamp, 'Asia/Taipei')) AS session_date,
          beacon_uid,
          b.beacon AS beacon,
          b.device AS device,
          b.session_duration_sec session_duration,
          geo,
          max_member_status,
          is_seller,
          first_session_timestamp,
          funnel_e1,
          funnel_e2,
          funnel_interest,
          funnel_consideration,
          add_to_cart,
          funnel_purchase,
          e2__uniq_items,
          if(funnel ['view_item'] IS NULL, 0, funnel ['view_item'].event_cnt) view_item_cnt,
          e2__session_duration,
          interest__shop_list,
          interest__item_list,
          search__ter

## [] Adoption_Rate

In [108]:
event_name = 'view_feed'
page_name = str.capitalize(str.replace(event_name , 'view_' , ''))

final = data_.get_metric_data(mode= 'sql' ,
                 metric = {
                     'metric_main':{
                         'metric_name':f'{page_name}_Adoption_Rate',
                         'source':'tracking',
                         'group_level':'user',
                         'method':'boolean',
                         'metric':{event_name:'view_id'}
                     }    
                          })
print(final)

WITH expt AS
  (SELECT beacon,
          exp_id,
          var_id,
          date(created) AS created_date
   FROM ath_experiment
   WHERE exp_id = 'beq_video_content_exp'
     AND date(created) <= date '2024-8-18' ),
     buyer_journey AS
  (SELECT visit_date,
          first_session_timestamp,
          date(at_timezone(first_session_timestamp, 'Asia/Taipei')) AS session_date,
          beacon_uid,
          b.beacon AS beacon,
          b.device AS device,
          b.session_duration_sec session_duration,
          geo,
          max_member_status,
          is_seller,
          first_session_timestamp,
          funnel_e1,
          funnel_e2,
          funnel_interest,
          funnel_consideration,
          add_to_cart,
          funnel_purchase,
          e2__uniq_items,
          if(funnel ['view_item'] IS NULL, 0, funnel ['view_item'].event_cnt) view_item_cnt,
          e2__session_duration,
          interest__shop_list,
          interest__item_list,
          search__ter

## avg.pl_ad_click_cnt per User

In [112]:
final = data_.get_metric_data(mode= 'sql' ,
                 metric = {
                     'metric_main':{
                         'metric_name':f'Avg.pl_ad_click_cnt per User',
                         'source':'tracking',
                         'group_level':'user',
                         'method':'count_distinct',
                         'condition':{
                             'view_item':{
                                 'promoted_type':'pl'
                             },
                             'view_shop':{
                                 'promoted_type':'pl'
                             }
                         },
                         'metric':{'view_item':'view_id'}
                     }    
                          })
print(final)

WITH expt AS
  (SELECT beacon,
          exp_id,
          var_id,
          date(created) AS created_date
   FROM ath_experiment
   WHERE exp_id = 'beq_video_content_exp'
     AND date(created) <= date '2024-8-18' ),
     buyer_journey AS
  (SELECT visit_date,
          first_session_timestamp,
          date(at_timezone(first_session_timestamp, 'Asia/Taipei')) AS session_date,
          beacon_uid,
          b.beacon AS beacon,
          b.device AS device,
          b.session_duration_sec session_duration,
          geo,
          max_member_status,
          is_seller,
          first_session_timestamp,
          funnel_e1,
          funnel_e2,
          funnel_interest,
          funnel_consideration,
          add_to_cart,
          funnel_purchase,
          e2__uniq_items,
          if(funnel ['view_item'] IS NULL, 0, funnel ['view_item'].event_cnt) view_item_cnt,
          e2__session_duration,
          interest__shop_list,
          interest__item_list,
          search__ter

## avg.pb_ad_click_cnt per User

In [113]:
final = data_.get_metric_data(mode= 'sql' ,
                 metric = {
                     'metric_main':{
                         'metric_name':f'Avg.pl_ad_click_cnt per User',
                         'source':'tracking',
                         'group_level':'user',
                         'method':'count_distinct',
                         'condition':{
                             'view_item':{
                                 'promoted_type':'pb'
                             },
                             'view_shop':{
                                 'promoted_type':'pb'
                             }
                         },
                         'metric':{'view_item':'view_id'}
                     }    
                          })
print(final)

WITH expt AS
  (SELECT beacon,
          exp_id,
          var_id,
          date(created) AS created_date
   FROM ath_experiment
   WHERE exp_id = 'beq_video_content_exp'
     AND date(created) <= date '2024-8-18' ),
     buyer_journey AS
  (SELECT visit_date,
          first_session_timestamp,
          date(at_timezone(first_session_timestamp, 'Asia/Taipei')) AS session_date,
          beacon_uid,
          b.beacon AS beacon,
          b.device AS device,
          b.session_duration_sec session_duration,
          geo,
          max_member_status,
          is_seller,
          first_session_timestamp,
          funnel_e1,
          funnel_e2,
          funnel_interest,
          funnel_consideration,
          add_to_cart,
          funnel_purchase,
          e2__uniq_items,
          if(funnel ['view_item'] IS NULL, 0, funnel ['view_item'].event_cnt) view_item_cnt,
          e2__session_duration,
          interest__shop_list,
          interest__item_list,
          search__ter

# For params

In [38]:
temp = pd.read_csv('../../../Downloads/sqllab_untitled_query_83_20240906T142600.csv')
temp = temp.query('~value_type.isin(["parse_error: no suffix" , "double_value" , "parse_error"])').drop(136)
temp.reset_index(drop = True , inplace = True)

In [44]:
temp.to_csv('params.csv' , index = False)